In [3]:
import pandas as pd
import re
import numpy as np

In [4]:
df = pd.read_csv("DF_BOOTCAMPS_2591.csv")

In [5]:
#CONSERVAR A TODOS AQUELLOS QUE NO TENGAN MISSING VALUES EN EDUCATION Y CERTIFICATION. 

dfe = df[df["education"].notna()] #DATAFRAME DE EDUCATION
dfe = dfe.reset_index(drop = True) #REINICIAR INDICE

dfc = df[df["certifications"].notna()] #DATAFRAME DE CERTIFICACIÓN

In [6]:
dfe.columns

Index(['id', 'scraping_services_id', 'linkedin_id', 'linkedin_profile_url',
       'last_name', 'first_name', 'v_extension', 'location', 'job_title',
       'company', 'job_history', 'snapshots', 'skills', 'education', 'about',
       'updated', 'middle_name', 'num_connections', 'interests', 'volunteered',
       'certifications', 'languages', 'recommendations', 'publications',
       'patents', 'industry', 'titles', 'company_name_complete',
       'job_title_complete'],
      dtype='object')

In [7]:
dfe.shape

(2566, 29)

In [8]:
# REGEX DE BOOTCAMPS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'dev f\\.'
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
 'ijalti', 
 'iron.?hack',
 'j.venes[ -]?a[ -]?programar',
 'keep.?coding',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'make[ -]?it[ -]?real[ -]?camp| make[ -]?it[ -]?real',
 'mind.?hub',
 'muktek|muk.?tek', 
 'nivel[ -]?pro ', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'programa[ -]?valentina',
 'reprograma',
 'worldtech', 
 'bictia|bi.?tia', 
 'talento[ -]?digital']

# EXTRAER FORMACIÓN DESDE EL DATASET DE LOS NO-MISSING VALUES DE EDUCACIÓN

In [9]:
#CONVERTIR EN STRING
dfe["education"] =  dfe["education"].str.lower()

In [10]:
#CONVERTIR EN DICCIONARIOS LA EDUCACIÓN
L = []

for i in dfe["education"]:
    l = str(i).split("},") #convertir en string la formación del individuo y crear lista para separar cada formación
    k = [] #Lista donde se almacenara cada formación x del individuo i
    for formacion in l: #para cada formación
        formacion = formacion.replace("[", "").replace("]", "") + "}" #separelas y conviertalas en diccionarios
        k.append(formacion) #añadalas a la lista de formación
    L.append(k) #añada la lista de formación para individuo

#VER UN EJEMPLO: Esta es la última formación del primer individuo desordenada en diccionario pero como un string   
L[0][0]

"{'education_org': 'university of oxford', 'degrees': 'university of oxford – oxford royale academy, english and culture, common european framework of reference: profiecient user', 'end': '2013', 'start': '2013'}"

In [11]:
K = [] #LISTA D ELISTAS EN DONDE SE ALMACENARAN LAS FORMACIONES A MANERA DE DICCIONARIO PARA CADA INDIVIDUO

for individuo in L: #para cada individuo

    educacion_ind = [] #creo una lista de sus educaciones

    for ins in individuo: # para cada cada formación dentro de las formaciones de cada individuos
        ins = ins.replace('"', "'")

        c = {}

        #CADA CAMPO SE EXTRAE POR MEDIO DE RegEx, COMUNES A TODOS LAS FORMACIONES STR DE LOS INDIVIDUOS EN EL DATASET
        #CADA CAMPO SE EXTRAE EN CASO DE QUE EXISTA DE LO CONTRARIO NO

        #EXTRAER EDUCACIÓN
        if ( len(re.findall("'education_org': '.*?'", ins)) ) > 0:
            c["education"] =  re.findall("'education_org': '.*?'", ins)[0].replace("'education_org': ", "").replace("'", "")

        #EXTRAER DEGREE
        if ( len(re.findall("'degrees': '.*?'", ins)) ) > 0:
            c["degrees"] =  re.findall("'degrees': '.*?'", ins)[0].replace("'degrees': ", "").replace("'", "")

        #EXTRAER SUMMARY
        if ( len(re.findall("'summary': '.*?'", ins)) ) > 0:
            c["summary"] =  re.findall("'summary': '.*?'", ins)[0].replace("'summary': ", "").replace("'", "")


        #EXTRAER INICIO
        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")

        #EXTRAER FINALIZACIÓN
        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")

        educacion_ind.append(c) #añado cada formación a la lista de sus educaciones
    
    K.append(educacion_ind) #añado la educación de cada individuo a una lista general

#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
K[0][0]

{'education': 'university of oxford',
 'degrees': 'university of oxford – oxford royale academy, english and culture, common european framework of reference: profiecient user',
 'start': '2013',
 'end': '2013'}

In [12]:
#CREAR COLUMNA LIMPIA DE EDUCACIÓN
dfe["educacion"] = K

In [14]:
#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
dfe.educacion[0][2]

{'education': 'vrije universiteit amsterdam',
 'degrees': '(pre)-master, organization science',
 'summary': '(schakeljaar tussen hbo-opleiding en masterstudie)\\n afstudeeronderzoek bij het ministerie van sociale zaken en werkgelegenheid(szw)\\n onderzoek op welke manier(en) de invoering van ‘het nieuwe werken’ bij het ministerie van szw, invloed heeft (gehad) op de betrokkenheid van de werknemers bij het uitvoeren hun werkzaamheden.\\n activities and societies: lid van faculteitsvereniging eos',
 'start': '2011',
 'end': '2012'}

In [114]:
# REGEX DE BOOTCAMPS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
 'ijalti', 
 'iron.?hack',
 'j.venes[ -]?a[ -]?programar',
 'keep.?coding|keep.?coding.io',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'make[ -]?it[ -]?real[ -]?camp| make[ -]?it[ -]?real',
 'mind.?hub',
 'muktek|muk.?tek', 
 'nivel[ -]?pro ', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'programa[ -]?valentina',
 'reprograma',
 'worldtech$',  #se cambió  para que solo termine con $
 'bictia|bi.?tia', 
 'talento[ -]?digital$'] #

 #VERDE: MÁS GRANDES DE LA, #AMARILLOS: MEDIANOS
 #EN PRINCIPIOS LOS VERDES SON LOS QUE DEBERIAN APARECER MÁS EN LOS PERFILES, EN PRINCIPIO.


 #BASE: LINKED-IN: + DIFICIL PARA HACER SCRAPING. EMPRESAS VENDEN DATOS (OBTUVIERON HACIENDO SCRAPING), PERFILES EGRESADOS DE BOOTCAMPS DE LISTA DE BOOTCAMPS EN LA. 3 RONDAS DE INTENTOS Y MANDAN MUESTRA PEQUEÑA. SUGIEREN MUCHOS DE LOS CAMBIOS.abs

 #NEGOCIACIÓN, PERFILES Y CUÁNTO CUESTA. MAX 8MIL. CADA FALSO POSITIVO ES UN PROBLEMA. 
 # LLEGABAN A MIL, PARECIA QUE FP NO ERAN PROBLEMA: MEJOR ESTRATEGIA PALABRAS GENERALES Y TRAEMOS 5MIL PERFILES Y DENTRO DE ESA DEBEN DE ESTAR TODOS ESTUDIANTES DE PERFILES PROFESIONALES DE BOOTCAMPS.
 # EGRESADOS EN LATIRNOAMERICA DEBERÍAN SER MUCHO MÁS DE 8MIL, EJ: DIGITAL HOUSE HABLAN DE 100MIL EGRESADOS

 #NO REPRESENTATIVIDAD: MUCHOS EGRESADOS POCA MUESTRA, NO TODOS TIENEN PERFIL DE LINKED-IN

 #PRIORIDAD

In [15]:
#LOOP PARA BUSCAR SI EL INDIVIDUO TIENE FORMACIÓN EN ALGÚN BOOTCAMP
c = []
m = []
for boot in boots:
    for i in range(len(dfe.educacion)):
        for edu in dfe.educacion[i]:
            # print(edu, i)
            if "education" in edu:
                # print(boot, edu["education"])
                A = re.findall(boot, edu["education"])
                if len(A) > 0:
                    c.append(A)
                    m.append(boot)
                    print(f" {dfe['job_title'][i]} -- {A} -- {boot} -- {edu['education']} \n \n")
                    # print(f"{A} -- {boot} -- {edu['education']}")

 CEO -- ['acamica'] -- ac.mica -- acamica 
 

 Co-founder & CEO -- ['acamica'] -- ac.mica -- acamica 
 

 Web Full Stack Developer -- ['acamica'] -- ac.mica -- acamica 
 

 Ingeniero -- ['acamica'] -- ac.mica -- acamica 
 

 Senior Javascript Developer -- ['acamica'] -- ac.mica -- codecademy/acamica 
 

 Technical Support Engineer -- ['acamica'] -- ac.mica -- acamica 
 

 Lider de Ecommerce -- ['acamica'] -- ac.mica -- acamica 
 

 API - UI Automation -- ['acamica'] -- ac.mica -- globant & acamica 
 

 Coordinadora de Marketing Digital -- ['acamica'] -- ac.mica -- acamica.com 
 

 Student -- ['acamica'] -- ac.mica -- acamica 
 

 Student -- ['acamica'] -- ac.mica -- acamica 
 

 Solution Developer. Frontend Developer (Angular) -- ['acamica'] -- ac.mica -- acamica 
 

 Solution Developer. Frontend Developer (Angular) -- ['acamica'] -- ac.mica -- acamica 
 

 UX/UI Designer -- ['acamica'] -- ac.mica -- acamica 
 

 UX/UI Designer -- ['acamica'] -- ac.mica -- acamica 
 

 UX/UI Designer -

In [116]:
len(c)

110